In [7]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import iterators, training, optimizers, serializers
from chainer.training import extensions

In [3]:
(train, test) = chainer.datasets.get_cifar10()
print("訓練データ数: ", len(train))
print("テストデータ数: ", len(test))

訓練データ数:  50000
テストデータ数:  10000


In [16]:
class Cifar10Model(chainer.Chain):

    def __init__(self, n_out=10):
        super(Cifar10Model,self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 32, 3, pad=1)
            self.conv2 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv3 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv4 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv5 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv6 = L.Convolution2D(32, 32, 3, pad=1)
            self.l1 = L.Linear(512, 512)
            self.l2 = L.Linear(512, n_out)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(F.relu(self.conv4(h)), 2)
        h = F.relu(self.conv5(h))
        h = F.max_pooling_2d(F.relu(self.conv6(h)), 2)
        h = F.dropout(F.relu(self.l1(h)))
        return self.l2(h)

In [22]:
class MyNet(chainer.Chain):

    def __init__(self, n_out=10):
        super(MyNet, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 32, 3, 3, 1)
            self.conv2 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv3 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv4 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv5 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv6 = L.Convolution2D(32, 32, 3, 3, 1)
            self.fc1 = L.Linear(None, 512)
            self.fc2 = L.Linear(512, n_out)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(F.relu(self.conv4(h)), 2)
        h = F.relu(self.conv5(h))
        h = F.max_pooling_2d(F.relu(self.conv6(h)), 2)
        h = F.dropout(F.relu(self.fc1(h)))
        h = self.fc2(h)
        return h

In [23]:
batch_size = 128
epoch = 1

#model = L.Classifier(Cifar10Model())
model = L.Classifier(MyNet())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

train_iter = chainer.iterators.SerialIterator(train, batch_size)
test_iter = chainer.iterators.SerialIterator(test, batch_size ,repeat=False, shuffle=False)

updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (epoch, 'epoch'), out="logs")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))                                                                                          
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.99875     1.76195               0.228001       0.324862                  
